In [ ]:
import numpy as np
import pandas as pd
import csv
import glob
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from collections import Counter
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [12]:
import numpy as np

# Class accuracies
accuracies = [89.51, 52.89, 98.13, 91.36, 97.03, 90.29, 98.31]

# Support (number of instances) for each class
support = [100, 200, 150, 120, 180, 130, 170]

# Calculate the weighted average accuracy
weighted_accuracy = np.average(accuracies, weights=support)

print(f"Weighted Average Accuracy: {weighted_accuracy:.2f}")

Weighted Average Accuracy: 89.31


In [2]:
import numpy as np

# Given accuracies and support for each label
accuracies = [0.1, 0.9, 0.3, 0.4, 0.8, 0.6, 0.7]
support = [50, 100, 30, 70, 90, 60, 40]

# Calculate weighted average accuracy
weighted_accuracy = np.average(accuracies, weights=support)

print(f"Weighted Average Accuracy: {weighted_accuracy:.2f}")

Weighted Average Accuracy: 0.61


In [ ]:
path = r'C:\Users\ibmcs\Downloads\cicids2017\full'
all_files = glob.glob(path + "/*.csv")
li = []
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0, encoding='utf-8', low_memory=False)
    li.append(df)

df = pd.concat(li, axis=0, ignore_index=True)
# df['padding1'] = 0
# df['padding2'] = 0
# df['padding3'] = 0
# df['padding4'] = 0
df

In [ ]:
df.columns = df.columns.str.lower().str.lstrip().str.rstrip().str.replace(" ","_")
df.replace([np.inf, -np.inf], np.nan, inplace=True)
# Drop rows with NaN
df.dropna(inplace=True)
df.columns

In [ ]:
all_obj_to_str = list(df.dtypes[df.dtypes == 'object'].index)
all_obj_to_str
for col in all_obj_to_str:
  df[col] = df[col].astype(str).str.lower()
df[col]

In [ ]:
df.columns

In [ ]:
# df.drop(['subflow_fwd_packets'],axis=1, inplace = True)
# df

In [ ]:
df['label'].value_counts()

In [ ]:
Y = df['label']
X = df.drop('label', axis = 1)
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, shuffle = True, random_state=42, stratify = y)
Y.value_counts()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, shuffle = True, random_state=42, stratify = Y)

In [ ]:
df_train = pd.concat([x_train, y_train], axis=1)
df_train

In [ ]:
df_less_freq_attack = df_train[df_train['label'].str.contains('bot|web|brute|dos|ben')]
df_less_freq_attack['label'].value_counts()

In [ ]:
y_df_less_freq_attack = df_less_freq_attack['label']
x_df_less_freq_attack = df_less_freq_attack.drop('label', axis = 1)
y_df_less_freq_attack

In [ ]:
# from imblearn.over_sampling import SMOTE
# from imblearn.over_sampling import SVMSMOTE
from imblearn.combine import SMOTEENN
# from imblearn.combine import SMOTETomek
counter = Counter(y_df_less_freq_attack)
print("before",counter)
# # Resampling the minority class. The strategy can be changed as required.
# sm = SMOTE()
# sm = SVMSMOTE(sampling_strategy='auto', random_state=None)
sm = SMOTEENN(sampling_strategy='auto')
# sm = SMOTETomek(random_state = 139)
# x_train,y_train = sm.fit_resample(x_train, y_train) 
x_df_less_freq_attack,y_df_less_freq_attack = sm.fit_resample(x_df_less_freq_attack,y_df_less_freq_attack)
# x_train,y_train = smotomek.fit_resample(x_train,y_train)

counter = Counter(y_df_less_freq_attack)
print("After",counter)

In [ ]:
df_less = pd.concat([x_df_less_freq_attack, y_df_less_freq_attack], axis=1)
df_less

In [ ]:
df_train = df_train[df_train.label != "dos"]
df_train = df_train[df_train.label != "benign"]
# df_train = df_train[df_train.label != "portscan"]
df_train = df_train[df_train.label != "brute_force"]
df_train = df_train[df_train.label != "web_attack"]
df_train = df_train[df_train.label != "bot"]

df_train['label'].value_counts()

In [ ]:
df_train = pd.concat([df_train, df_less], axis=0)
df_train['label'].unique()

In [ ]:
y_train = df_train['label']
x_train = df_train.drop('label', axis = 1)
# x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.15, shuffle = True, random_state=42, stratify = Y)
# # x_train,valid_x,train_label,valid_label = train_test_split(x_train, y_train, test_size=0.20, random_state=42, stratify = y_train)
y_train.value_counts()

In [ ]:
print('Training data shape : ', x_train.shape, y_train.shape)
print('Testing data shape : ', x_test.shape, y_test.shape)
# Find the unique numbers from the train labels
classes = np.unique(y_train)
nClasses = len(classes)
print('Total number of outputs : ', nClasses)
print('Output classes : ', classes)

In [ ]:
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)
print(x_train.shape)
print(x_test.shape)

In [ ]:
#label encoding y_train
label_encoder = preprocessing.LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.fit_transform(y_test)

In [ ]:
#label encoding y_train
# label_encoder = preprocessing.LabelEncoder()
y_test = to_categorical(y_test,  num_classes = 7)
y_train = to_categorical(y_train, num_classes = 7)
y_test.shape, y_train.shape

In [ ]:
#Reshape
x_train = x_train.reshape(-1,7,7)
x_test = x_test.reshape(-1,7,7)
print("x_train shape: ",x_train.shape)
print("x_test shape: ",x_test.shape)

In [ ]:
from sklearn.model_selection import train_test_split
x_train,valid_x,train_label,valid_label = train_test_split(x_train, y_train, test_size=0.15, random_state=42, shuffle=True, stratify = y_train)

In [ ]:
x_train.shape[1:],valid_x.shape,train_label.shape,valid_label.shape

In [ ]:
import keras
from keras.models import Sequential,Input,Model
from keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional, Flatten, CuDNNLSTM, Bidirectional
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
# from tensorflow.keras.layers import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from tensorflow.keras.optimizers import RMSprop,Adam, SGD
import tensorflow as tf 
from sklearn.utils import class_weight
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
batch_size = 64
epochs = 50
num_classes = 7

In [ ]:
model1 = Sequential()
model1.add(Conv2D(16, kernel_size=(2, 2),activation='relu',padding='same',input_shape=(7,7,1)))
model1.add(Conv2D(16, (2, 2), activation='relu',padding='same'))
# model1.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
# model1.add(MaxPooling2D(pool_size=(2, 2),padding='same'))

# model1.add(Dropout(0.25))


model1.add(Conv2D(32, (2, 2), activation='relu',padding='same'))
model1.add(Conv2D(32, (2, 2), activation='relu',padding='same'))
# model.add(Conv2D(64, (2, 2), activation='relu',padding='same'))
model1.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
# model1.add(MaxPooling2D(pool_size=(2, 2),padding='same'))

# model.add(Dropout(0.20))


model1.add(Conv2D(64, (2, 2), activation='relu',padding='same'))
model1.add(Conv2D(64, (2, 2), activation='relu',padding='same'))
# model.add(Conv2D(128, (2, 2), activation='relu',padding='same'))
# model.add(Conv2D(256, (3, 3), activation='relu',padding='same'))
model1.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
# model1.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
# model1.add(Dropout(0.20))




model1.add(Flatten())
model1.add(Dense(1024, activation='relu'))       
model1.add(Dropout(0.3))
model1.add(Dense(512, activation='relu'))
model1.add(Dense(num_classes, activation='softmax'))

In [ ]:
optimizer = Adam(learning_rate=0.001)
model1.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

In [ ]:
history1 = model1.fit(x_train, train_label, batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(valid_x, valid_label),shuffle=True)

In [ ]:
# !pip install scikeras
# from scikeras.wrappers import KerasClassifier, KerasRegressor
# NN_clf1=KerasClassifier(build_fn=model_cnn, epochs=5, batch_size= 64)

In [ ]:
# %pip install h5py
# model_cnn.save('C:/Users/ibmcs/Downloads/UNSW-NB15/model_cnn.h5')
model1.save('model1.h5')

In [ ]:
x_train.shape[1:3]

In [ ]:
#model lstm
# def model_lstm ():
model2 = Sequential()
model2.add(CuDNNLSTM(16, input_shape=(x_train.shape[1:3]),  return_sequences=True))
model2.add(Dropout(0.2))
# Second LSTM layer
# model.add(CuDNNLSTM(128))
model2.add(CuDNNLSTM(32,  return_sequences=False))
model2.add(Dropout(0.2))
# model.add(LSTM(128, activation = "relu"))


model2.add(Flatten())
model2.add(Dense(32, activation = "relu"))
model2.add(Dropout(0.5))
model2.add(Dense(16, activation = "relu"))
# The output layer
model2.add(Dense(7, activation = "softmax"))

  # return model2

In [ ]:
optimizer = Adam(learning_rate=0.001)
model2.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

In [ ]:
history2 = model2.fit(x_train, train_label, batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(valid_x, valid_label),shuffle=True)

In [ ]:
# from scikeras.wrappers import KerasClassifier, KerasRegressor
# NN_clf2=KerasClassifier(build_fn=model_lstm, epochs=5, batch_size= 64)

In [ ]:
model2.save('model2.h5')

In [ ]:
# def model_gru():
model3 = Sequential()
model3.add(GRU(16, input_shape=(x_train.shape[1:3]), activation='tanh', return_sequences=True))

model3.add(Dropout(0.2))


model3.add(GRU(32, activation='tanh'))
model3.add(Dropout(0.2))

model3.add(Flatten())
model3.add(Dense(32, activation='relu'))       
model3.add(Dropout(0.5))
model3.add(Dense(16, activation='relu'))
model3.add(Dense(num_classes, activation='softmax'))

  # return model3


In [ ]:
optimizer = Adam(learning_rate=0.001)
model3.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

In [ ]:
history3 = model3.fit(x_train, train_label, batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(valid_x, valid_label),shuffle=True)

In [ ]:
# from scikeras.wrappers import KerasClassifier, KerasRegressor
# NN_clf3=KerasClassifier(build_fn=model_gru, epochs=5, batch_size= 64)

In [ ]:
# intermediate = [('NN1', NN_clf1), ('NN2', NN_clf2), ('NN3', NN_clf3)]
# intermediate

In [ ]:
# from sklearn.ensemble import StackingClassifier
# from sklearn.neural_network import MLPClassifier
# clf = StackingClassifier(estimators=intermediate, final_estimator=MLPClassifier())
# train_label.reshape(-1)
# train_label.shape
# clf.fit(x_train, train_label)

In [ ]:
model3.save('model3.h5')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
epochs = range(1,50)
plt.plot(history1.history['val_accuracy'], color='r', label="validation accuracy")
plt.plot(history1.history['accuracy'], color='b', label="training accuracy")
plt.title('Training and Validation Accuracy LSTM')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
epochs = range(1,50)
plt.plot(history1.history['val_loss'], color='r', label="validation loss")
plt.plot(history1.history['loss'], color='b', label="training loss")
plt.title('Training and Validation loss LSTM')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
# Predict the values from the validation dataset
Y_pred = model1.predict(valid_x)
sns.set(font_scale=1.5) # Adjust to fit
# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
# Convert validation observations to one hot vectors
Y_true = np.argmax(valid_label,axis = 1) 
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 
# plot the confusion matrix
f,ax = plt.subplots(figsize=(10, 9))
cmn = confusion_mtx.astype('float') / confusion_mtx.sum(axis=1)[:, np.newaxis]
class_names = ['Benign', 'Bot', 'Brute-Force','Dos',
               'DDoS', 'Portscan', 'Webattack']
sns.heatmap(cmn, annot=True, linewidths=0.02,cmap="Reds",linecolor="gray", fmt= '.2f',xticklabels=class_names, yticklabels=class_names)
# label_font = {'size':'5'}  # Adjust to fit
# title_font = {'size':'5'}  # Adjust to fit

plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix of CICIDS-2017 for Proposed ENIDS")
plt.show()

In [ ]:
#Recall Score
from sklearn.metrics import recall_score

recall = recall_score(Y_true, Y_pred_classes, labels=[0,1,2,3,4,5,6], average='weighted')

print('Recall: %.5f' % recall)

In [ ]:
#Precision Score
from sklearn.metrics import precision_score
precision = precision_score(Y_true, Y_pred_classes,labels=[0,1,2,3,4,5,6], average='weighted')
print('Recall: %.5f' % precision)

In [ ]:
from sklearn.metrics import f1_score
f1_m = f1_score(Y_true, Y_pred_classes, labels=[0,1,2,3,4,5,6], average='weighted')
print('F-Measure: %.5f' % f1_m)

In [ ]:
from sklearn import metrics
# Print the confusion matrix
Y_pred = model1.predict(x_train)
Y_pred_classes = np.argmax(Y_pred,axis = 1)
Y_true = np.argmax(train_label, axis=1)
print(metrics.confusion_matrix(Y_true, Y_pred_classes))

# Print the precision and recall, among other metrics
print(metrics.classification_report(Y_true, Y_pred_classes, digits=5))

In [ ]:
from sklearn import metrics
predicted_classes = model1.predict(x_test)
predicted_classes = np.argmax(np.round(predicted_classes),axis= 1)

Y_test = np.argmax(y_test, axis=1)
print(predicted_classes.shape)
print(y_test.shape)

# Print the confusion matrix
print(metrics.confusion_matrix(Y_test, predicted_classes))

# Print the precision and recall, among other metrics
print(metrics.classification_report(Y_test, predicted_classes, digits=4))

In [ ]:
# x_test.shape, y_test.shape
test_eval = model1.evaluate(x_test, y_test, verbose= 1 )
print( 'Test loss:' , test_eval[0])
print( 'Test accuracy:' , test_eval[1])

In [ ]:
model1.fit(x_test, y_test)
Y_pred =model1.predict(x_test)
predicted_classes = np.argmax(np.round(Y_pred),axis= 1)

Y_test = np.argmax(y_test, axis=1)
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix
confusion_matrix(Y_test, predicted_classes) 
plt.show()
print(classification_report(Y_test, predicted_classes, digits=5))

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
# Predict the values from the validation dataset
Y_pred = model1.predict(x_test)
# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
# Convert validation observations to one hot vectors
Y_true = np.argmax(y_test,axis = 1) 
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 
# plot the confusion matrix
f,ax = plt.subplots(figsize=(8, 8))
cmn = confusion_mtx.astype('float') / confusion_mtx.sum(axis=1)[:, np.newaxis]
class_names = ['Benign', 'Bot', 'Brute-force','Dos',
               'DDoS', 'Portscan', 'Webattack']
sns.heatmap(cmn, annot=True, linewidths=0.02,cmap="Reds",linecolor="gray", fmt= '.2f',xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix CNN")
plt.show()

In [ ]:
from keras.layers.merge import concatenate
from keras.utils.vis_utils import plot_model
from keras.models import Model
from tensorflow.keras.models import Model
from keras.layers import Input
from keras.models import load_model

In [ ]:
dependencies = {
    'f1_m': f1_m,
    'recall_m': recall,
    'precision': precision
}

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
dependencies = {
    'f1_m': f1_score,
    'recall_m': recall_score,
    'precision': precision_score
}

In [ ]:
# load models from file
def load_all_models(n_models):
	all_models = list()
	for i in range(n_models):
		# define filename for this ensemble
		filename = 'model' + str(i + 1) + '.h5'
		# load model from file
		model = load_model(filename,custom_objects=dependencies)
		# add to list of members
		all_models.append(model)
		print('>loaded %s' % filename)
	return all_models

In [ ]:
n_members = 3
members = load_all_models(n_members)
print('Loaded %d models' % len(members))

In [ ]:
# define stacked model from multiple member input models
def define_stacked_model(members):
	# update all layers in all models to not be trainable
	for i in range(len(members)):
		model = members[i]
		for layer in model.layers:
			# make not trainable
			layer.trainable = False
			# rename to avoid 'unique layer name' issue
			layer._name = 'ensemble_' + str(i+1) + '_' + layer.name
	# define multi-headed input
	ensemble_visible = [model.input for model in members]
	# concatenate merge output from each model
	ensemble_outputs = [model.output for model in members]
	merge = concatenate(ensemble_outputs)
	print(merge.shape)
# 	model.add(Flatten())
# fit standalone model
	flatten = Flatten()(merge)
	hidden = Dense(1024, activation='relu')(flatten)
	hidden = Dense(512, activation='relu')(hidden)
	# print(hidden.shape)
	output = Dense(7, activation='softmax')(hidden)
	# print(output.shape)
	model = Model(inputs=ensemble_visible, outputs=output)
	# print(model.shape)
	# plot graph of ensemble
	plot_model(model, show_shapes=True, to_file='model_graph.png')
	# compile
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
# Create the stacked model
stacked_model = define_stacked_model(members)

In [ ]:
define_stacked_model.summary()

In [ ]:
# define ensemble model
stacked_model = define_stacked_model(members)

In [ ]:
aug = tf.keras.preprocessing.image.ImageDataGenerator(
          rotation_range=10,
          zoom_range=0.15,
          width_shift_range=0.1,
          height_shift_range=0.1,
          shear_range=0.15,
          horizontal_flip= False,
          vertical_flip= False,
          fill_mode="nearest"
          )

In [ ]:

# fit a stacked model
def fit_stacked_model(model, inputX, inputy):
	# prepare input data
	X = [inputX for _ in range(len(model.input))]
	# encode output data
	# inputy_enc = to_categorical(inputy)
	# fit model
	
	model.fit(X, inputy, epochs=50, batch_size=64, verbose=1, shuffle=True)
#  history3 = model3.fit(x_train, train_label, batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(valid_x, valid_label),shuffle=True)
  

In [ ]:

# fit a stacked model
def fit_stacked_model(stacked_model, inputX, inputy):
	# prepare input data
	X = [inputX for _ in range(len(stacked_model.input))]
	# encode output data
	# inputy_enc = to_categorical(inputy)
	# fit model
	
	stacked_model.fit(X, inputy, epochs=50, batch_size=64, verbose=1, shuffle=True)
#  history3 = model3.fit(x_train, train_label, batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(valid_x, valid_label),shuffle=True)
	return stacked_model

In [ ]:
from tensorflow.keras.models import save_model

# fit a stacked model
def fit_stacked_model(stacked_model, inputX, inputy, filename='stack_ensemble.h5'):
    # prepare input data
    X = [inputX for _ in range(len(stacked_model.input))]
    # encode output data
    # inputy_enc = to_categorical(inputy)
    # fit model
    stacked_model.fit(X, inputy, epochs=30, batch_size=128, verbose=1, shuffle=True)
    # save the model
    save_model(stacked_model, filename)
    return stacked_model

In [ ]:
# fit stacked model on test dataset
# print(stacked_model)
# x_train.shape, train_label.shape
# train_label = y_train = tf.keras.utils.to_categorical(train_label)
fit_stacked_model(stacked_model, x_train, train_label)

In [ ]:
# make a prediction with a stacked model
def predict_stacked_model(stacked_model, inputX):
	# prepare input data
	X = [inputX for _ in range(len(stacked_model.input))]
	# make prediction
	return stacked_model.predict(X, verbose=1)

In [ ]:
from sklearn import metrics
predicted_classes = predict_stacked_model(stacked_model, x_test)
predicted_classes = np.argmax(np.round(predicted_classes),axis= 1)

Y_test = np.argmax(y_test, axis=1)
print(predicted_classes.shape)
print(y_test.shape)

# Print the confusion matrix
print(metrics.confusion_matrix(Y_test, predicted_classes))

# Print the precision and recall, among other metrics
print(metrics.classification_report(Y_test, predicted_classes, digits=3))
# make predictions and evaluate
# import numpy as np  
# from sklearn.metrics import accuracy_score

# yhat = predict_stacked_model(stacked_model, x_test)
# yhat = np.argmax(yhat, axis=1)
# acc = accuracy_score(y_test, yhat)
# print('Stacked Test Accuracy: %.3f' % acc)

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
# Predict the values from the validation dataset
# Y_pred = predict_stacked_model.predict(stacked_model, x_test)
# print(Y_pred)
# Convert predictions classes to one hot vectors 
# Y_pred_classes = np.argmax(Y_pred,axis = 1) 
# Convert validation observations to one hot vectors
# Y_true = np.argmax(valid_label,axis = 1) 
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_test, predicted_classes) 
# plot the confusion matrix
f,ax = plt.subplots(figsize=(8, 8))
cmn = confusion_mtx.astype('float') / confusion_mtx.sum(axis=1)[:, np.newaxis]
class_names = ['benign', 'bot', 'brute-force','Dos',
               'DDoS', 'Portscan', 'Webattack']
sns.heatmap(cmn, annot=True, linewidths=0.02,cmap="Reds",linecolor="gray", fmt= '.2f',xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
from sklearn.linear_model import LogisticRegression
optimizer = Adam(learning_rate=0.001)
# model_cnn_lstm.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

In [ ]:
# fit a model based on the outputs from the ensemble members
def fit_stacked_model(members, inputX, inputy):
	# create dataset using ensemble
	stackedX = stacked_dataset(members, inputX)
	stackedX = stackedX.reshape((-1, 3,7))
	# fit standalone model
	model_cnn_lstm = Sequential()
	model_cnn_lstm.add(Conv2D(32, kernel_size=(2, 2),padding='same',input_shape=(3,7,1)))
	model_cnn_lstm.add(MaxPooling2D(pool_size=(2,2), padding  ="same"))
	model_cnn_lstm.add(Dropout(0.15))

	model_cnn_lstm.add(Conv2D(64, (2, 2), activation='relu',padding='same'))
	model_cnn_lstm.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
	model_cnn_lstm.add(Dropout(0.15))

	model_cnn_lstm.add(Conv2D(128, (2, 2), activation='relu',padding='same'))           
	model_cnn_lstm.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
	model_cnn_lstm.add(Dropout(0.15))



	model_cnn_lstm.add(Flatten())
	# model_cnn_lstm.add(Reshape((-1, 128)))
	#insert lstm

	# model_cnn_lstm.add(LSTM(64,return_sequences=True))
	model_cnn_lstm.add(Dense(1024, activation='relu'))        
	model_cnn_lstm.add(Dropout(0.3))
	# model_cnn_lstm.add(LSTM(64,return_sequences=False))
	model_cnn_lstm.add(Dense(512, activation='relu'))       
	# model_cnn_lstm.add(Dropout(0.5))
	# model_cnn_lstm.add(Dense(512, activation='relu'))
	model_cnn_lstm.add(Dense(num_classes, activation='softmax'))
	# model_cnn_lstm = LogisticRegression() #meta learner
	model_cnn_lstm.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])
	# history3 = model3.fit(x_train, train_label, batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(valid_x, valid_label),shuffle=True)
	# model_cnn_lstm.fit(stackedX, inputy, batch_size=64,epochs=5,verbose=1,validation_data=(valid_x, valid_label),shuffle=True)
	model_cnn_lstm.fit(stackedX, inputy,batch_size=64,epochs=5,verbose=1)
	# model_cnn_lstm.summary()

	return model_cnn_lstm

In [ ]:
# print(x_train.shape)
# print(y_train.shape)
model_cnn_lstm = fit_stacked_model(members, x_test,y_test)
# model_cnn_lstm.summary()

In [ ]:
# make a prediction with the stacked model
def stacked_prediction(members, model_cnn_lstm, inputX):
	# create dataset using ensemble
	stackedX = stacked_dataset(members, inputX)
	stackedX =stackedX.reshape(-1,3,7)
	# make a prediction
	yhat = model_cnn_lstm.predict(stackedX)
	return yhat

In [ ]:
# evaluate model on test set
# print(valid_x.shape)
yhat = stacked_prediction(members, model_cnn_lstm, valid_x)
from sklearn.metrics import f1_score
f1_m = f1_score(Y_true, Y_pred_classes, labels=[0,1,2,3,4,5,6], average='weighted')
print('F-Measure: %.3f' % f1_m)

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
i = 0
for model in members:
    i+=1
    pred = model.predict_classes(X_test)
    score = f1_score(y_test,pred)
    print('F-Score of model {} is '.format(i),score)